In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import PIL
import PIL.Image
from tensorflow import keras
from keras import layers
from keras.models import Sequential

In [9]:
batch_size = 32
img_height = 80
img_width = 200

class_names = ['fahtema', 'saf', 'mom']

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  './dataset',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4005 files belonging to 3 classes.
Using 3204 files for training.


In [4]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  './dataset',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4005 files belonging to 3 classes.
Using 801 files for validation.


In [5]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [6]:
normalization_layer = layers.Rescaling(1./255)

In [7]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [10]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [11]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [24]:
print(train_ds)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 80, 200, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [12]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10
101/101 [==============================] - 15s 143ms/step - loss: 0.8727 - accuracy: 0.5824 - val_loss: 0.4466 - val_accuracy: 0.8065
Epoch 2/10
101/101 [==============================] - 13s 133ms/step - loss: 0.4389 - accuracy: 0.8230 - val_loss: 0.3475 - val_accuracy: 0.8652
Epoch 3/10
101/101 [==============================] - 14s 134ms/step - loss: 0.3570 - accuracy: 0.8555 - val_loss: 0.3050 - val_accuracy: 0.8764
Epoch 4/10
101/101 [==============================] - 13s 132ms/step - loss: 0.3095 - accuracy: 0.8761 - val_loss: 0.2907 - val_accuracy: 0.8764
Epoch 5/10
101/101 [==============================] - 13s 132ms/step - loss: 0.2771 - accuracy: 0.8895 - val_loss: 0.2753 - val_accuracy: 0.8876
Epoch 6/10
101/101 [==============================] - 14s 136ms/step - loss: 0.2534 - accuracy: 0.8983 - val_loss: 0.2513 - val_accuracy: 0.8951
Epoch 7/10
101/101 [==============================] - 14s 135ms/step - loss: 0.2433 - accuracy: 0.9042 - val_loss: 0.3248 - val_ac

In [14]:
model.save('mymodel')

INFO:tensorflow:Assets written to: mymodel\assets


INFO:tensorflow:Assets written to: mymodel\assets


In [15]:
trainedModel = keras.models.load_model("mymodel")

In [18]:
img = tf.keras.utils.load_img(
    './dataset/fahtema/fahtema1004.png', target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 [==============================] - 0s 81ms/step
This image most likely belongs to fahtema with a 100.00 percent confidence.
